# **Notebook for Advanced Analysis of timeseries data**  
**Description:** Notebook for advanced analysis of snowpack diagnostic trends at both individual points and in the ensemble average including breakpoint detection.  
**Input data:** NetCDF file of problematic snow cover characteristic durations from Script4SnowDiagnostics.ipynb  
**Output data:** Visualizations and statistical analyses of problematic snow characteristic trends  
**Creator:** Emma Perkins  
**Date:** January 2024

In [ ]:
#import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import glob
import xarray as xr
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.api as sm
import os
from sklearn.preprocessing import PolynomialFeatures
import piecewise_regression
import re

In [ ]:
#read in the data
full_data = xr.open_dataset('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/NetCDF_files/NoTree/1950_2100_md350_wc01_iv70_start_end.nc')
full_avg = full_data.mean(dim = 'point').mean(dim = 'run')

#split off future-term for separate analysis (2029 - 2100)
focus_data = full_data.sel(season = range(2029,2100))
focus_avg = focus_data.mean(dim = 'point').mean(dim = 'run')

## **Plotting Function for single diagnostic**

In [ ]:
def piecewise_single(data, ax, var, pt):
    for i in range(10):
        ax.plot(data.sel(point = pt).sel(run = i + 1).season, data.sel(point = pt).sel(run = i + 1)[var].values, color = 'lightblue')
    x = data.sel(point = pt).mean(dim = 'run').season.values
    y = data.sel(point = pt).mean(dim = 'run')[var].values
    ax.plot(x, y, color = 'blue')
    pw_fit = piecewise_regression.Fit(x, y, n_breakpoints = 1, n_boot = 10)
    mod_text = ' '
    if pw_fit.davies < 0.05: #if the data pass the davies test for the existence of at least one breakpoint, plot the segmented regression
        #figure out how many breakpoints by choosing the model with the lowest BIC
        ms = piecewise_regression.ModelSelection(x, y, max_breakpoints=2)
        bic_list = []
        for i in range(3):
            bic_list.append(ms.model_summaries[i]['bic'])
        bic_array = np.array(bic_list, dtype = np.float)
        n = int(np.where(abs(bic_array) == min(abs(bic_array)))[0][0])
        if n > 0:
            #fit a regression with the chosen number of breakpoints
            pw_fit = piecewise_regression.Fit(x, y, n_breakpoints=n)
            ax.plot(x, pw_fit.predict(x), color = 'black', linewidth = 3)
            r_val = np.float(pw_fit.summary()[pw_fit.summary().find('R Squared'):pw_fit.summary().find('Adjusted R Squared') - 1].split(' ')[-1])
            mod_text = 'R^2 = '+str(np.round(r_val,2))
        else:
            X = sm.add_constant(x)
            mod = sm.OLS(y,X).fit()
            if (sms.linear_harvey_collier(mod)[1] > 0.05): #check if data meets assumtions for linear regression
                if mod.f_pvalue < 0.05: #if assumtions met, check if model is significant
                    ax.plot(x, mod.predict(), color = 'black', linewidth = 3) #if model is significant plot the regression line
                    mod_text = 'Lin R^2 = '+str(np.round(mod.rsquared,2))
                else:
                    mod_text = 'Linearity assumption met but trend not significant.'
            else: #if the data don't meet the assumptions for linear regression try quadratic regression
                polynomial_features= PolynomialFeatures(degree=2)
                X = polynomial_features.fit_transform(x.reshape(-1,1))
                mod = sm.OLS(y.reshape(-1,1), X).fit()
                if (sms.linear_harvey_collier(mod)[1] > 0.05): #check if second-order regression meets assumptions
                    if mod.f_pvalue < 0.05:
                        ax.plot(x, mod.predict(), color = 'black', linewidth = 3) #if second-order assumtions are met plot the regression
                        #mod_text = 'y = '+str(mod.params[0])+' + '+str(mod.params[1])+'x + '+str(mod.params[2])+'x^2'
                        mod_text = 'Quad R^2 = '+str(np.round(mod.rsquared,2))
                    else:
                        mod_text = 'Linearity assumption met but trend not significant.'
                else:
                    mod_text = 'Linearity assumption not met.'
    else: #if the data don't have a breakpoint use regular linear regression
        X = sm.add_constant(x)
        mod = sm.OLS(y,X).fit()
        if (sms.linear_harvey_collier(mod)[1] > 0.05): #check if data meets assumtions for linear regression
            if mod.f_pvalue < 0.05: #if assumtions met, check if model is significant
                ax.plot(x, mod.predict(), color = 'black', linewidth = 3) #if model is significant plot the regression line
                mod_text = 'Lin R^2 = '+str(np.round(mod.rsquared,2))
            else:
                mod_text = 'Linearity assumption met but trend not significant.'
        else: #if the data don't meet the assumptions for linear regression try quadratic regression
            polynomial_features= PolynomialFeatures(degree=2)
            X = polynomial_features.fit_transform(x.reshape(-1,1))
            mod = sm.OLS(y.reshape(-1,1), X).fit()
            if (sms.linear_harvey_collier(mod)[1] > 0.05): #check if second-order regression meets assumptions
                if mod.f_pvalue < 0.05:
                    ax.plot(x, mod.predict(), color = 'black', linewidth = 3) #if second-order assumtions are met plot the regression
                    #mod_text = 'y = '+str(mod.params[0])+' + '+str(mod.params[1])+'x + '+str(mod.params[2])+'x^2'
                    mod_text = 'Quad R^2 = '+str(np.round(mod.rsquared,2))
                else:
                    mod_text = 'Linearity assumption met but trend not significant.'
            else:
                mod_text = 'Linearity assumption not met.'
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    return mod_text

# **Individual Points**

### **Icy Days -- 1950 - 2100**

In [ ]:
for pt in range(1,9):    
    fig, ([ax1, ax2, ax3]) = plt.subplots(3,1, figsize = [20,30])
    
    #ice_5
    mod_text = piecewise_single(full_data, ax1, 'ice5', pt)
    ax1.set_ylabel('Instances of 5 Consecutive Icy Days', fontsize = 15, fontweight = 'bold')
    ax1.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    ax1.set_title('Point '+str(pt)+' Icy Day Characteristics 1950 - 2100', fontsize = 20, fontweight = 'bold')
    ax1.text(0.7, 0.9, mod_text, transform=ax1.transAxes, fontsize = 20, fontweight = 'bold')
    #ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
    #ax1.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')

    
    #ice_count_max
    mod_text = piecewise_single(full_data, ax2, 'ice_count_max', pt)
    ax2.set_ylabel('Maximum Duration of Continuous Ice (days)', fontsize = 15, fontweight = 'bold')
    ax2.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    #ax2.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax2.text(0.7, 0.9, mod_text, transform=ax2.transAxes, fontsize = 20, fontweight = 'bold')
    
    #ice_total
    mod_text = piecewise_single(full_data, ax3, 'ice_total', pt)
    ax3.set_ylabel('Total Number of Icy Days', fontsize = 15, fontweight = 'bold')
    ax3.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    #ax3.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax3.text(0.7, 0.9, mod_text, transform=ax3.transAxes, fontsize = 20, fontweight = 'bold')
    
    #plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/1950_2100/Point'+str(pt)+'ice_chars.png')

### **Icy Days -- 2029 - 2100**

In [ ]:
for pt in range(1,9):    
    fig, ([ax1, ax2, ax3]) = plt.subplots(3,1, figsize = [20,30])
    
    #ice_5
    mod_text = piecewise_single(focus_data, ax1, 'ice5', pt)
    ax1.set_ylabel('Instances of 5 Consecutive Icy Days', fontsize = 15, fontweight = 'bold')
    ax1.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    ax1.set_title('Point '+str(pt)+' Icy Day Characteristics 2029 - 2100', fontsize = 20, fontweight = 'bold')
    ax1.text(0.7, 0.9, mod_text, transform=ax1.transAxes, fontsize = 20, fontweight = 'bold')
    #ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
    #ax1.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')

    
    #ice_count_max
    mod_text = piecewise_single(focus_data, ax2, 'ice_count_max', pt)
    ax2.set_ylabel('Maximum Duration of Continuous Ice (days)', fontsize = 15, fontweight = 'bold')
    ax2.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    #ax2.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax2.text(0.7, 0.9, mod_text, transform=ax2.transAxes, fontsize = 20, fontweight = 'bold')
    
    #ice_total
    mod_text = piecewise_single(focus_data, ax3, 'ice_total', pt)
    ax3.set_ylabel('Total Number of Icy Days', fontsize = 15, fontweight = 'bold')
    ax3.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    #ax3.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax3.text(0.7, 0.9, mod_text, transform=ax3.transAxes, fontsize = 20, fontweight = 'bold')
    
    #plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/1950_2100/Point'+str(pt)+'ice_chars.png')

### **Deep Days -- 1950 - 2100**

In [ ]:
for pt in range(1,9):    
    fig, ([ax1, ax2, ax3]) = plt.subplots(3,1, figsize = [20,30])
    
    #ice_5
    mod_text = piecewise_single(full_data, ax1, 'deep5', pt)
    ax1.set_ylabel('Instances of 5 Consecutive Deep Days', fontsize = 15, fontweight = 'bold')
    ax1.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    ax1.set_title('Point '+str(pt)+' Deep Day Characteristics 1950 - 2100', fontsize = 20, fontweight = 'bold')
    ax1.text(0.7, 0.9, mod_text, transform=ax1.transAxes, fontsize = 20, fontweight = 'bold')
    #ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
    #ax1.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')

    
    #ice_count_max
    mod_text = piecewise_single(full_data, ax2, 'deep_count_max', pt)
    ax2.set_ylabel('Maximum Duration of Continuous Deep (days)', fontsize = 15, fontweight = 'bold')
    ax2.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    #ax2.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax2.text(0.7, 0.9, mod_text, transform=ax2.transAxes, fontsize = 20, fontweight = 'bold')
    
    #ice_total
    mod_text = piecewise_single(full_data, ax3, 'deep_total', pt)
    ax3.set_ylabel('Total Number of Deep Days', fontsize = 15, fontweight = 'bold')
    ax3.set_xlabel('Season', fontsize = 15, fontweight = 'bold')
    #ax3.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax3.text(0.7, 0.9, mod_text, transform=ax3.transAxes, fontsize = 20, fontweight = 'bold')
    
    #plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/1950_2100/Point'+str(pt)+'ice_chars.png')

### **Deep Days -- 2029 - 2100**

In [ ]:
for pt in range(1,9):    
    fig, ([ax1, ax2, ax3]) = plt.subplots(3,1, figsize = [20,30])
    
    #ice_5
    mod_text = piecewise_single(focus_data, ax1, 'deep5', pt)
    ax1.set_ylabel('Instances of 5 Consecutive Deep Days', fontsize = 15, fontweight = 'bold')
    ax1.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    ax1.set_title('Point '+str(pt)+' Deep Day Characteristics 2029 - 2100', fontsize = 20, fontweight = 'bold')
    ax1.text(0.7, 0.9, mod_text, transform=ax1.transAxes, fontsize = 20, fontweight = 'bold')
    #ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
    #ax1.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')

    
    #ice_count_max
    mod_text = piecewise_single(focus_data, ax2, 'deep_count_max', pt)
    ax2.set_ylabel('Maximum Duration of Continuous Deep (days)', fontsize = 15, fontweight = 'bold')
    ax2.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    #ax2.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax2.text(0.7, 0.9, mod_text, transform=ax2.transAxes, fontsize = 20, fontweight = 'bold')
    
    #ice_total
    mod_text = piecewise_single(focus_data, ax3, 'deep_total', pt)
    ax3.set_ylabel('Total Number of Deep Days', fontsize = 15, fontweight = 'bold')
    ax3.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    #ax3.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax3.text(0.7, 0.9, mod_text, transform=ax3.transAxes, fontsize = 20, fontweight = 'bold')
    
    #plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/1950_2100/Point'+str(pt)+'ice_chars.png')

### **Season Length -- 1950 - 2100**

In [ ]:
for pt in range(1,9):    
    fig, ([ax1, ax2, ax3]) = plt.subplots(3,1, figsize = [20,30])
    
    #ice_5
    mod_text = piecewise_single(full_data, ax1, 'length', pt)
    ax1.set_ylabel('Snow Year Length', fontsize = 15, fontweight = 'bold')
    ax1.set_xlabel('Snow year', fontsize = 15, fontweight = 'bold')
    ax1.set_title('Point '+str(pt)+' Length Characteristics 1950 - 2100', fontsize = 20, fontweight = 'bold')
    ax1.text(0.7, 0.9, mod_text, transform=ax1.transAxes, fontsize = 20, fontweight = 'bold')
    #ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
    #ax1.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')

    
    #ice_count_max
    mod_text = piecewise_single(full_data, ax2, 'season_start', pt)
    ax2.set_ylabel('Snow Year Start Date', fontsize = 15, fontweight = 'bold')
    ax2.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    #ax2.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax2.text(0.7, 0.9, mod_text, transform=ax2.transAxes, fontsize = 20, fontweight = 'bold')
    
    #ice_total
    mod_text = piecewise_single(full_data, ax3, 'season_end', pt)
    ax3.set_ylabel('Snow Year End Date', fontsize = 15, fontweight = 'bold')
    ax3.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    #ax3.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax3.text(0.7, 0.9, mod_text, transform=ax3.transAxes, fontsize = 20, fontweight = 'bold')
    
    #plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/1950_2100/Point'+str(pt)+'ice_chars.png')

### **Season Length -- 2029 - 2100**

In [ ]:
for pt in range(1,9):    
    fig, ([ax1, ax2, ax3]) = plt.subplots(3,1, figsize = [20,30])
    
    #ice_5
    mod_text = piecewise_single(focus_data, ax1, 'length', pt)
    ax1.set_ylabel('Snow Year Length', fontsize = 15, fontweight = 'bold')
    ax1.set_xlabel('Snow year', fontsize = 15, fontweight = 'bold')
    ax1.set_title('Point '+str(pt)+' Length Characteristics 1950 - 2100', fontsize = 20, fontweight = 'bold')
    ax1.text(0.7, 0.9, mod_text, transform=ax1.transAxes, fontsize = 20, fontweight = 'bold')
    #ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
    #ax1.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')

    
    #ice_count_max
    mod_text = piecewise_single(focus_data, ax2, 'season_start', pt)
    ax2.set_ylabel('Snow Year Start Date', fontsize = 15, fontweight = 'bold')
    ax2.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    #ax2.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax2.text(0.7, 0.9, mod_text, transform=ax2.transAxes, fontsize = 20, fontweight = 'bold')
    
    #ice_total
    mod_text = piecewise_single(focus_data, ax3, 'season_end', pt)
    ax3.set_ylabel('Snow Year End Date', fontsize = 15, fontweight = 'bold')
    ax3.set_xlabel('Snow Year', fontsize = 15, fontweight = 'bold')
    #ax3.text(2080,text_y, mod_text, fontsize = 15, fontweight = 'bold')
    ax3.text(0.7, 0.9, mod_text, transform=ax3.transAxes, fontsize = 20, fontweight = 'bold')
    
    #plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/1950_2100/Point'+str(pt)+'ice_chars.png')

# **Ensemble Averages**

### **Season Length Diagnostics - All (1950 - 2100 and 2029 - 2100)**

In [ ]:
r_vals = []

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize = [40,30])

mod_text = piecewise_plotter(full_data, ax1, 'length')
#ax1.text(0.8,0.9, mod_text, transform=ax1.transAxes, fontsize = 30, fontweight = 'bold')
ax1.set_ylabel('Snow Year Length (days)', fontsize = 50, fontweight = 'bold')
ax1.set_title('1950 - 2100', fontsize = 50, fontweight = 'bold')
ax1.set_xticks([])
ax1.set_yticklabels(ax1.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(full_data, ax3, 'season_start')
#ax3.text(0.8,0.9, mod_text, transform=ax3.transAxes, fontsize = 30, fontweight = 'bold')
ax3.set_ylabel('Snow Year Start Date', fontsize = 50, fontweight = 'bold')
ax3.set_xticks([])
ax3.set_yticklabels(ax3.get_yticks(), fontsize = 30)
r_vals.append(mod_text)
    
mod_text = piecewise_plotter(full_data, ax5, 'season_end')
#ax5.text(0.8,0.9, mod_text, transform=ax5.transAxes, fontsize = 30, fontweight = 'bold')
ax5.set_ylabel('Snow Year End Date', fontsize = 50, fontweight = 'bold')
ax5.set_xlabel('Snow Year', fontsize = 50, fontweight = 'bold')
ax5.set_xticklabels(ax5.get_xticks(), fontsize = 30)
ax5.set_yticklabels(ax5.get_yticks(), fontsize = 30)
r_vals.append(mod_text)
        
mod_text = piecewise_plotter(focus_data, ax2, 'length')
#ax2.text(0.8,0.9, mod_text, transform=ax2.transAxes, fontsize = 30, fontweight = 'bold')
ax2.set_title('2029 - 2100', fontsize = 50, fontweight = 'bold')
ax2.set_xticks([])
ax2.set_yticklabels(ax2.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(focus_data, ax4, 'season_start')
#ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
ax4.set_xticks([])
ax4.set_yticklabels(ax4.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(focus_data, ax6, 'season_end')
#ax6.text(0.8,0.9, mod_text, transform=ax6.transAxes, fontsize = 30, fontweight = 'bold')
ax6.set_xlabel('Snow Year', fontsize = 50, fontweight = 'bold')
ax6.set_xticklabels(ax6.get_xticks(), fontsize = 30)
ax6.set_yticklabels(ax6.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

ax1.text(0.02, 0.01, 'a', transform=ax1.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax2.text(0.02, 0.01, 'b', transform=ax2.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax3.text(0.02, 0.01, 'c', transform=ax3.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax4.text(0.02, 0.01, 'd', transform=ax4.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax5.text(0.02, 0.01, 'e', transform=ax5.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax6.text(0.02, 0.01, 'f', transform=ax6.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')

        
fig.suptitle('Snow Year Length Per Point', fontsize = 60, fontweight = 'bold')
plt.tight_layout()
    
plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/NoTree/LE_mean_length_breakpoints_no_R2.png')

### **Season Length Diagnostics -- 1950 - 2100 only**

In [ ]:
r_vals_season = []

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = [20,30])

mod_text = piecewise_plotter(full_data, ax1, 'length')
#ax1.text(0.8,0.9, mod_text, transform=ax1.transAxes, fontsize = 30, fontweight = 'bold')
ax1.set_ylabel('Snow Year Length (days)', fontsize = 40, fontweight = 'bold')
ax1.set_title('Snow Year Length Per Point 1950 - 2100', fontsize = 50, fontweight = 'bold')
ax1.set_xticks([])
ax1.set_yticklabels(ax1.get_yticks(), fontsize = 30)
r_vals_season.append(mod_text)

mod_text = piecewise_plotter(full_data, ax2, 'season_start')
#ax3.text(0.8,0.9, mod_text, transform=ax3.transAxes, fontsize = 30, fontweight = 'bold')
ax2.set_ylabel('Snow Year Start Date', fontsize = 40, fontweight = 'bold')
ax2.set_xticks([])
ax2.set_yticklabels(ax2.get_yticks(), fontsize = 30)
r_vals_season.append(mod_text)
    
mod_text = piecewise_plotter(full_data, ax3, 'season_end')
#ax5.text(0.8,0.9, mod_text, transform=ax5.transAxes, fontsize = 30, fontweight = 'bold')
ax3.set_ylabel('Snow Year End Date', fontsize = 40, fontweight = 'bold')
ax3.set_xlabel('Snow Year', fontsize = 40, fontweight = 'bold')
ax3.set_xticklabels(ax3.get_xticks(), fontsize = 30)
ax3.set_yticklabels(ax3.get_yticks(), fontsize = 30)
r_vals_season.append(mod_text)

ax1.text(0.02, 0.01, 'a', transform=ax1.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
ax2.text(0.02, 0.01, 'b', transform=ax2.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
ax3.text(0.02, 0.01, 'c', transform=ax3.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
        
#fig.suptitle('Snow Year Length Per Point 1950 - 2100', fontsize = 60, fontweight = 'bold')
plt.tight_layout()
    
plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/NoTree/LE_mean_length_breakpoints_no_R2_1950_2100.png')
print(r_vals_season)

### **Icy Day Diagnostics - All (1950 - 2100 and 2029 - 2100)**

In [ ]:
r_vals = []

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize = [40,30])

mod_text = piecewise_plotter(full_data, ax1, 'ice5')
#ax1.text(0.8,0.9, mod_text, transform=ax1.transAxes, fontsize = 30, fontweight = 'bold')
ax1.set_ylabel('Ice5 (count)', fontsize = 50, fontweight = 'bold')
ax1.set_title('1950 - 2100', fontsize = 50, fontweight = 'bold')
ax1.set_xticks([])
ax1.set_yticklabels(ax1.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(full_data, ax3, 'ice_count_max')
#ax3.text(0.8,0.9, mod_text, transform=ax3.transAxes, fontsize = 30, fontweight = 'bold')
ax3.set_ylabel('Ice Count Max (days)', fontsize = 50, fontweight = 'bold')
ax3.set_xticks([])
ax3.set_yticklabels(ax3.get_yticks(), fontsize = 30)
r_vals.append(mod_text)
    
mod_text = piecewise_plotter(full_data, ax5, 'ice_total')
#ax5.text(0.8,0.9, mod_text, transform=ax5.transAxes, fontsize = 30, fontweight = 'bold')
ax5.set_ylabel('Ice Total (days)', fontsize = 50, fontweight = 'bold')
ax5.set_xlabel('Snow Year', fontsize = 50, fontweight = 'bold')
ax5.set_xticklabels(ax5.get_xticks(), fontsize = 30)
ax5.set_yticklabels(ax5.get_yticks(), fontsize = 30)
r_vals.append(mod_text)
        
mod_text = piecewise_plotter(focus_data, ax2, 'ice5')
#ax2.text(0.8,0.9, mod_text, transform=ax2.transAxes, fontsize = 30, fontweight = 'bold')
ax2.set_title('2029 - 2100', fontsize = 50, fontweight = 'bold')
ax2.set_xticks([])
ax2.set_yticklabels(ax2.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(focus_data, ax4, 'ice_count_max')
#ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
ax4.set_xticks([])
ax4.set_yticklabels(ax4.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(focus_data, ax6, 'ice_total')
#ax6.text(0.8,0.9, mod_text, transform=ax6.transAxes, fontsize = 30, fontweight = 'bold')
ax6.set_xlabel('Snow Year', fontsize = 50, fontweight = 'bold')
ax6.set_xticklabels(ax6.get_xticks(), fontsize = 30)
ax6.set_yticklabels(ax6.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

ax1.text(0.02, 0.01, 'a', transform=ax1.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax2.text(0.02, 0.01, 'b', transform=ax2.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax3.text(0.02, 0.01, 'c', transform=ax3.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax4.text(0.02, 0.01, 'd', transform=ax4.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax5.text(0.02, 0.01, 'e', transform=ax5.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax6.text(0.02, 0.01, 'f', transform=ax6.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')

        
fig.suptitle('Icy Diagnostics Per Point', fontsize = 60, fontweight = 'bold')
plt.tight_layout()
    
plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/LE_mean_ice_breakpoints.png')

### **Icy Day Diagnostics -- 1950 - 2100 only**

In [ ]:
r_vals_ice = []

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = [20,30])

mod_text = piecewise_plotter(full_data, ax1, 'ice5')
ax1.set_ylabel('Ice5 (count)', fontsize = 40, fontweight = 'bold')
ax1.set_title('Icy Diagnostics Per Point 1950 - 2100', fontsize = 50, fontweight = 'bold')
ax1.set_xticks([])
ax1.set_yticklabels(ax1.get_yticks(), fontsize = 30)
r_vals_ice.append(mod_text)

mod_text = piecewise_plotter(full_data, ax2, 'ice_count_max')
ax2.set_ylabel('Ice Count Max (days)', fontsize = 40, fontweight = 'bold')
ax2.set_xticks([])
ax2.set_yticklabels(ax2.get_yticks(), fontsize = 30)
r_vals_ice.append(mod_text)
    
mod_text = piecewise_plotter(full_data, ax3, 'ice_total')
ax3.set_ylabel('Ice Total (days)', fontsize = 40, fontweight = 'bold')
ax3.set_xlabel('Snow Year', fontsize = 40, fontweight = 'bold')
ax3.set_xticklabels(ax3.get_xticks(), fontsize = 30)
ax3.set_yticklabels(ax3.get_yticks(), fontsize = 30)
r_vals_ice.append(mod_text)

ax1.text(0.02, 0.01, 'a', transform=ax1.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
ax2.text(0.02, 0.01, 'b', transform=ax2.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
ax3.text(0.02, 0.01, 'c', transform=ax3.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
        
plt.tight_layout()
    
plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/NoTree/LE_mean_ice_breakpoints_no_R2_1950_2100.png')
print(r_vals_ice)

### **Deep Day Diagnostics - All (1950 - 2100 and 2029 - 2100)**

In [ ]:
r_vals = []

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize = [40,30])

mod_text = piecewise_plotter(full_data, ax1, 'deep5')
#ax1.text(0.8,0.9, mod_text, transform=ax1.transAxes, fontsize = 30, fontweight = 'bold')
ax1.set_ylabel('Deep5 (count)', fontsize = 50, fontweight = 'bold')
ax1.set_title('1950 - 2100', fontsize = 50, fontweight = 'bold')
ax1.set_xticks([])
ax1.set_yticklabels(ax1.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(full_data, ax3, 'deep_count_max')
#ax3.text(0.8,0.9, mod_text, transform=ax3.transAxes, fontsize = 30, fontweight = 'bold')
ax3.set_ylabel('Deep Count Max (days)', fontsize = 50, fontweight = 'bold')
ax3.set_xticks([])
ax3.set_yticklabels(ax3.get_yticks(), fontsize = 30)
r_vals.append(mod_text)
    
mod_text = piecewise_plotter(full_data, ax5, 'deep_total')
#ax5.text(0.8,0.9, mod_text, transform=ax5.transAxes, fontsize = 30, fontweight = 'bold')
ax5.set_ylabel('Deep Total (days)', fontsize = 50, fontweight = 'bold')
ax5.set_xlabel('Snow Year', fontsize = 50, fontweight = 'bold')
ax5.set_xticklabels(ax5.get_xticks(), fontsize = 30)
ax5.set_yticklabels(ax5.get_yticks(), fontsize = 30)
r_vals.append(mod_text)
        
mod_text = piecewise_plotter(focus_data, ax2, 'deep5')
#ax2.text(0.8,0.9, mod_text, transform=ax2.transAxes, fontsize = 30, fontweight = 'bold')
ax2.set_title('2029 - 2100', fontsize = 50, fontweight = 'bold')
ax2.set_xticks([])
ax2.set_yticklabels(ax2.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(focus_data, ax4, 'deep_count_max')
#ax4.text(0.8,0.9, mod_text, transform=ax4.transAxes, fontsize = 30, fontweight = 'bold')
ax4.set_xticks([])
ax4.set_yticklabels(ax4.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

mod_text = piecewise_plotter(focus_data, ax6, 'deep_total')
#ax6.text(0.8,0.9, mod_text, transform=ax6.transAxes, fontsize = 30, fontweight = 'bold')
ax6.set_xlabel('Snow Year', fontsize = 50, fontweight = 'bold')
ax6.set_xticklabels(ax6.get_xticks(), fontsize = 30)
ax6.set_yticklabels(ax6.get_yticks(), fontsize = 30)
r_vals.append(mod_text)

ax1.text(0.02, 0.01, 'a', transform=ax1.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax2.text(0.02, 0.01, 'b', transform=ax2.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax3.text(0.02, 0.01, 'c', transform=ax3.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax4.text(0.02, 0.01, 'd', transform=ax4.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax5.text(0.02, 0.01, 'e', transform=ax5.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')
ax6.text(0.02, 0.01, 'f', transform=ax6.transAxes, fontsize=20, fontweight='bold', va='bottom', ha='right')

        
fig.suptitle('Deep Diagnostics Per Point', fontsize = 60, fontweight = 'bold')
plt.tight_layout()
    
plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/LE_mean_deep_breakpoints.png')

### **Deep Day Diagnostics -- 1950 - 2100 only**

In [ ]:
r_vals_deep = []

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = [20,30])

mod_text = piecewise_plotter(full_data, ax1, 'deep5')
ax1.set_ylabel('Deep5 (count)', fontsize = 40, fontweight = 'bold')
ax1.set_title('Deep Diagnostics Per Point 1950 - 2100', fontsize = 50, fontweight = 'bold')
ax1.set_xticks([])
ax1.set_yticklabels(ax1.get_yticks(), fontsize = 30)
r_vals_deep.append(mod_text)

mod_text = piecewise_plotter(full_data, ax2, 'deep_count_max')
ax2.set_ylabel('Deep Count Max (days)', fontsize = 40, fontweight = 'bold')
ax2.set_xticks([])
ax2.set_yticklabels(ax2.get_yticks(), fontsize = 30)
r_vals_deep.append(mod_text)
    
mod_text = piecewise_plotter(full_data, ax3, 'deep_total')
ax3.set_ylabel('Deep Total (days)', fontsize = 40, fontweight = 'bold')
ax3.set_xlabel('Snow Year', fontsize = 40, fontweight = 'bold')
ax3.set_xticklabels(ax3.get_xticks(), fontsize = 30)
ax3.set_yticklabels(ax3.get_yticks(), fontsize = 30)
r_vals_deep.append(mod_text)

ax1.text(0.02, 0.01, 'a', transform=ax1.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
ax2.text(0.02, 0.01, 'b', transform=ax2.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
ax3.text(0.02, 0.01, 'c', transform=ax3.transAxes, fontsize=30, fontweight='bold', va='bottom', ha='right')
        
plt.tight_layout()
    
plt.savefig('/glade/work/eperkins/SNOWPACK_data/Results/analysis/GE_alts/plots/NoTree/LE_mean_deep_breakpoints_no_R2_1950_2100.png')
print(r_vals_deep)